## IMPORTANT:
initial trajectory is important for final solution, with good initial trajectory it finds solution really easy. This suggests to use the old trajectory initial input to iLQG algorithm

In [1]:
import numpy as np
from drl.ilqg import ilqg, LearnedDynamics
from drl.env.arm import TwoLinkArm

env = TwoLinkArm(g=0., wp=10., wv=1., wu=0.001)

N = 5 # number of future steps for iLQG
Nf = 2 # number of time-steps for fitting linear model
num_episodes = 25
max_steps = 50

model = LearnedDynamics(max_steps, num_episodes, env.state_dim, env.action_dim, Nf)

In [2]:
def dyn(x, u):
    return model.dynamics_function(x, u)

In [ ]:
def cst(x, u):
    return env.cost_func(x, u)

In [ ]:
x = x0 = env.reset()
goal = env.goal

# Initialize random control sequence
u = np.random.randn(max_steps, env.action_dim)

# Simulate system once
reward = 0.
for i_step in range(max_steps):
    env.render()
    x_new, r, t, _ = env.step(u[i_step,:]) 
    
    model.add(0, i_step, x, u[i_step,:], x_new)
    
    x = x_new    
    reward += r
print('Iter %d, Steps %d, Reward: %.2f, Average reward: %.2f' % (0, i_step+1, reward, reward/i_step))

# Only use first N control inputs for iLQG estimator
u = u[:N,:]
    
for i_episode in range(1, num_episodes):
    # Fit models
    model.fit()
    
    x = env.reset(x0, goal)
    terminal = False
    i_step = 0
    reward = 0.
    
    for i_step in range(max_steps):
        env.render()
        
        model.set_cur_step(i_step)

        _, u, L, Vx, Vxx, cost = ilqg(dyn, cst, x, u, {})
                
        # Take step
        x_new, r, t, _ = env.step(u[0, :])

        # Add to data matrices
        model.add(i_episode, i_step, x, u[0, :], x_new)
        
        u = np.concatenate((u[1:,:], np.random.randn(1, env.action_dim))) 
        
        x = x_new
        reward += r
        i_step += 1
        
        if t:
            break
    
    print('Iter %d, Steps %d, Reward: %.2f, Average reward: %.2f' % (i_episode, i_step, reward, reward/i_step))

Iter 0, Steps 50, Reward: -964.94, Average reward: -19.69
Iter 1, Steps 50, Reward: -707.18, Average reward: -14.14
Iter 2, Steps 50, Reward: -628.82, Average reward: -12.58
Iter 3, Steps 50, Reward: -502.47, Average reward: -10.05
Iter 4, Steps 46, Reward: -494.14, Average reward: -10.74
Iter 5, Steps 45, Reward: -399.01, Average reward: -8.87
Iter 6, Steps 49, Reward: -531.17, Average reward: -10.84
Iter 7, Steps 43, Reward: -464.95, Average reward: -10.81
Iter 8, Steps 41, Reward: -445.60, Average reward: -10.87
Iter 9, Steps 41, Reward: -433.47, Average reward: -10.57
Iter 10, Steps 40, Reward: -426.27, Average reward: -10.66


In [ ]:
env.render(close=True)